<a href="https://colab.research.google.com/github/illaho/INVEST_IO/blob/master/yahoo_finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 3.3MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.

In [2]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json
import pprint
import time

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver', options=options)

wd.get('https://it.finance.yahoo.com/')
# Click on 'I agree' button and wait for a second
wd.find_element_by_xpath("//button[@value='agree']").click()
time.sleep(1)

In [ ]:
print(wd.title)
wd.save_screenshot('screenshot.png')

%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/content/screenshot.png')
imgplot = plt.imshow(img)
plt.show()

In [5]:
tickers = ['AMZN', 'AAPL']
detail_news = []
for ticker in tqdm(tickers):
  wd.get(f"https://it.finance.yahoo.com/quote/{ticker}/news?p={ticker}")
  #wd.save_screenshot(f'screenshot_{ticker}.png')
  # Driver scrolls down three times to load the news
  for i in range(0,3):
    wd.execute_script("window.scrollBy(0,5000)")
    time.sleep(2)

  news_list = wd.find_elements_by_css_selector('#latestQuoteNewsStream-0-Stream > ul > li.js-stream-content')
  print(len(news_list))

  
  for news in news_list:
     _id = news.find_element_by_css_selector("h3 a").get_attribute("href").split("-")[-1].split(".")[0]
     author_press = news.find_element_by_css_selector("span").text
     title = news.find_element_by_css_selector("h3").text
     url = news.find_element_by_css_selector("h3 a").get_attribute("href")
     description = news.find_element_by_css_selector("p").text  
     detail_news.append({'_id': _id,
                          'ticker': ticker,
                          'title': title,
                          'description': description,
                          'url': url,
                          'author_press': author_press
                        })
    
print(len(detail_news))
pprint.pprint(detail_news)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


40
40

80
[{'_id': '124417307',
  'author_press': 'Askanews',
  'description': 'Indagine di 15 mesi. "Non c\'è riduzione della concorrenza"',
  'ticker': 'AMZN',
  'title': 'Deliveroo: via libera authority Gb a investimento di Amazon',
  'url': 'https://it.finance.yahoo.com/notizie/deliveroo-via-libera-authority-gb-investimento-di-amazon-124417307.html'},
 {'_id': '214718558',
  'author_press': 'Askanews',
  'description': 'Dow Jones +0,44, Nasdaq +1,49, il colosso di Cupertino fa '
                 '+10%',
  'ticker': 'AMZN',
  'title': 'Wall Street chiude settimana in positivo trainata da Apple e Tech',
  'url': 'https://it.finance.yahoo.com/notizie/wall-street-chiude-settimana-positivo-trainata-da-apple-214718558.html'},
 {'_id': '143116531',
  'author_press': 'Askanews',
  'description': 'Il provvedimento riguarder 200mila dipendenti',
  'ticker': 'AMZN',
  'title': 'Usa, Google estende lo smart working fino a giugno 2021',
  'url': 'https://it.finance.yahoo.com/notizie/usa-google-

In [6]:
ds_detail_news = pd.DataFrame(detail_news)
ds_detail_news.set_index("_id")
ds_detail_news.head()
ds_detail_news.tail()

,_id,ticker,title,description,url,author_press
75,112926313,AAPL,Tim: arriva il nuovo smartphone iPhone SE,Il prodotto è prenotabile online e sarà spedit...,https://it.finance.yahoo.com/notizie/tim-arriv...,Askanews
76,091821740,AAPL,"Coronavirus, Antitrust contro truffe online di...",Appello ai gestori dei principali motori di ri...,https://it.finance.yahoo.com/notizie/coronavir...,Askanews
77,164018024,AAPL,"Apple lancia l'iPhone low-cost, in Italia a pa...",L'iPhone SE di seconda generazione costerà 399...,https://it.finance.yahoo.com/notizie/apple-lan...,Askanews
78,201940029,AAPL,"Wall Street ottimista su coronavirus, Dow Jone...","Aiuta il rally dei titoli tecnologici, il Nasd...",https://it.finance.yahoo.com/notizie/wall-stre...,Askanews
79,100026589,AAPL,"Coronavirus, Fs lancia App per autocertificazi...",Permette di generare un file Pdf da stampare e...,https://it.finance.yahoo.com/notizie/coronavir...,Askanews


In [7]:
ds_detail_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           80 non-null     object
 1   ticker        80 non-null     object
 2   title         80 non-null     object
 3   description   80 non-null     object
 4   url           80 non-null     object
 5   author_press  80 non-null     object
dtypes: object(6)
memory usage: 3.9+ KB


In [8]:
ds_detail_news.to_csv('/ds_news_yahoo_finance.csv')

In [9]:
ds_detail_news = pd.read_csv('/ds_news_yahoo_finance.csv', index_col="_id")
ds_detail_news.head()

,Unnamed: 0,ticker,title,description,url,author_press
_id,,,,,,
124417307,0,AMZN,Deliveroo: via libera authority Gb a investime...,"Indagine di 15 mesi. ""Non c'è riduzione della ...",https://it.finance.yahoo.com/notizie/deliveroo...,Askanews
214718558,1,AMZN,Wall Street chiude settimana in positivo train...,"Dow Jones +0,44, Nasdaq +1,49, il colosso di C...",https://it.finance.yahoo.com/notizie/wall-stre...,Askanews
143116531,2,AMZN,"Usa, Google estende lo smart working fino a gi...",Il provvedimento riguarder 200mila dipendenti,https://it.finance.yahoo.com/notizie/usa-googl...,Askanews
120618670,3,AMZN,"Antitrust, Amazon: massima collaborazione all'...",Istruttoria per accertare intesa con Apple res...,https://it.finance.yahoo.com/notizie/antitrust...,Askanews
93718510,4,AMZN,Antitrust: indagine su Apple-Amazon per diviet...,Sui prodotti Apple-Beats ai rivenditori non ad...,https://it.finance.yahoo.com/notizie/antitrust...,Askanews


In [11]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np

details = []

for _id, news in ds_detail_news.iterrows(): 
    time.sleep(1 + np.random.randn()*0.2)
    link = news["url"]
    wd.set_window_size(1920, 1080)
    wd.get(link)
    date = wd.find_elements_by_css_selector("time.date")[0].text
    date_UTC = wd.find_elements_by_css_selector("time.date")[0].get_attribute("datetime")
    article = [] 
    container_news = wd.find_elements_by_css_selector("article div p")
    
    for p in range(len(container_news)):
      piece_of_article = wd.find_elements_by_css_selector("article div p")[p].text
      article.append(piece_of_article)
     
    details.append({'_id': _id,
                    'date': date,
                    'date_UTC': date_UTC,
                    'article': article
                    #'article_read_more': article_read_more
                    })
  
len(details)
pprint.pprint(details)

"""    try:
      for i in range(0,3):
        wd.execute_script("window.scrollBy(0,5000)")
        time.sleep(2)

      wait = WebDriverWait(wd, 60)
      wait.until(EC.presence_of_element_located((By.XPATH, '//span[text()="Continua a leggere"]')))
      driver.find_element_by_xpath('//span[text()="Continua a leggere"]').click  
      #WebDriverWait(wd, 5).until(
      #    EC.element_to_be_clickable((By.CSS_SELECTOR, "button.read-more > span"))//*[@id="tgt1-Col1-0-ContentCanvas"]/article/div/button/span
      #)
      #wd.find_element_by_css_selector("button.read-more > span").click()
      article_read_more = wd.find_elements_by_css_selector("div.Ov(h)")[0].text
    except: 
      article_read_more = ""
      

    date = wd.find_elements_by_css_selector("time.date")[0].text
    date_UTC = wd.find_elements_by_css_selector("time.date")[0].get_attribute("datetime")
    article = wd.find_elements_by_css_selector("article")[0].text
    for i in range(0,5):
        wd.execute_script("window.scrollBy(0,10_000)")
        time.sleep(2)
    
    if(len(wd.find_elements_by_css_selector("button.read-more")) >1):
      wd.find_element_by_css_selector("button.read-more").click()
      article_read_more = wd.find_elements_by_css_selector(".Oh(v) > p")[0].text
    else:
      article_read_more = ""

wd.get('https://it.finance.yahoo.com/notizie/primi-10-anni-di-amazon-nel-2020-previste-040222028.html')      
"""


[{'_id': 124417307,
  'article': ["Roma, 4 ago. (askanews) - Via libera all'investimento di Amazon "
              'in Deliveroo. Dopo 15 mesi di indagine, la Competition and '
              'markets authority (Cma) del Regno Unito ha emanato la decisione '
              "finale, confermando che l'investimento di minoranza del colosso "
              'di Seattle in Deliveroo può andare avanti.',
              'La Cma ha concluso che l\'investimento di minoranza "non ha '
              'determinato, e non si prevede che determini, una sostanziale '
              'riduzione della concorrenza". E sono state respinte le '
              "argomentazioni dei competitor in opposizione all'investimento.",
              'Deliveroo collabora adesso con 100mila ristoranti in '
              'piattaforma a livello globale. Trentamila ristoranti hanno '
              "aderito soltanto quest'anno, con la crisi del coronavirus che "
              'ha spinto i ristoratori a rivolgersi al delivery. E 75

'    try:\n      for i in range(0,3):\n        wd.execute_script("window.scrollBy(0,5000)")\n        time.sleep(2)\n\n      wait = WebDriverWait(wd, 60)\n      wait.until(EC.presence_of_element_located((By.XPATH, \'//span[text()="Continua a leggere"]\')))\n      driver.find_element_by_xpath(\'//span[text()="Continua a leggere"]\').click  \n      #WebDriverWait(wd, 5).until(\n      #    EC.element_to_be_clickable((By.CSS_SELECTOR, "button.read-more > span"))//*[@id="tgt1-Col1-0-ContentCanvas"]/article/div/button/span\n      #)\n      #wd.find_element_by_css_selector("button.read-more > span").click()\n      article_read_more = wd.find_elements_by_css_selector("div.Ov(h)")[0].text\n    except: \n      article_read_more = ""\n      \n\n    date = wd.find_elements_by_css_selector("time.date")[0].text\n    date_UTC = wd.find_elements_by_css_selector("time.date")[0].get_attribute("datetime")\n    article = wd.find_elements_by_css_selector("article")[0].text\n    for i in range(0,5):\n       

In [12]:
ds_article_news = pd.DataFrame(details)
ds_article_news.set_index("_id")
ds_article_news.head()

,_id,date,date_UTC,article
0,124417307,4 agosto 2020,2020-08-04T12:47:58.000Z,"[Roma, 4 ago. (askanews) - Via libera all'inve..."
1,214718558,31 luglio 2020,2020-07-31T21:51:28.000Z,"[Milano, 31 lug. (askanews) - La settimana di ..."
2,143116531,27 luglio 2020,2020-07-27T14:35:09.000Z,"[New York, 27 lug. (askanews) - Google ha este..."
3,120618670,22 luglio 2020,2020-07-22T12:10:49.000Z,"[Milano, 22 lug. (askanews) - ""Stiamo offrendo..."
4,93718510,22 luglio 2020,2020-07-22T09:42:11.000Z,"[Roma, 22 lug. (askanews) - Indagine Antitrust..."


In [13]:
ds_article_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   _id       80 non-null     int64 
 1   date      80 non-null     object
 2   date_UTC  80 non-null     object
 3   article   80 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.6+ KB


In [14]:
ds_article_news.to_csv('/ds_article_yahoo_finance.csv')

In [20]:
tickers = ['ISP', 'UBI', 'AZM', 'ANIM']
industry = []
key_figures = []
for ticker in tqdm(tickers):
  wd.get(f'https://finance.yahoo.com/quote/{ticker}.MI/profile?p={ticker}.MI')
  company_name = wd.find_element_by_css_selector("#Col1-0-Profile-Proxy > section > div.asset-profile-container > div > h3").text
  industry_ticker = wd.find_element_by_css_selector("#Col1-0-Profile-Proxy > section > div.asset-profile-container > div > div > p.D\(ib\).Va\(t\) > span:nth-child(5)").text
  industry.append({'ticker': ticker,
                   'company_name': company_name,
                   'industry_ticker': industry_ticker
  })
  apicals = wd.find_elements_by_css_selector("tbody tr")
  for apical in apicals:
    name = apical.find_elements_by_css_selector("td span")[0].text
    role = apical.find_elements_by_css_selector("td span")[1].text
    key_figures.append({'ticker': ticker,
                        'name': name,
                        'role': role})
pprint.pprint(industry)
pprint.pprint(key_figures)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.



[{'company_name': 'Intesa Sanpaolo S.p.A.',
  'industry_ticker': 'Banks—Regional',
  'ticker': 'ISP'},
 {'company_name': 'Unione di Banche Italiane S.p.A.',
  'industry_ticker': 'Banks—Regional',
  'ticker': 'UBI'},
 {'company_name': 'Azimut Holding S.p.A.',
  'industry_ticker': 'Asset Management',
  'ticker': 'AZM'},
 {'company_name': 'Anima Holding S.p.A.',
  'industry_ticker': 'Asset Management',
  'ticker': 'ANIM'}]
[{'name': 'Mr. Carlo Messina',
  'role': 'MD, CEO, GM & Exec. Director',
  'ticker': 'ISP'},
 {'name': 'Mr. Stefano Del Punta',
  'role': 'Group Chief Financial Officer',
  'ticker': 'ISP'},
 {'name': 'Ms. Paola Angeletti',
  'role': 'Chief Operating Officer',
  'ticker': 'ISP'},
 {'name': 'Mr. Massimo Proverbio',
  'role': 'Chief IT, Digital & Innovation Office',
  'ticker': 'ISP'},
 {'name': 'Fabrizio Dabbene',
  'role': 'Head of Admin. & Tax Department and Mang.',
  'ticker': 'ISP'},
 {'name': 'Ms. Letizia Maria Brichetto Arnaboldi Moratti',
  'role': 'Chairwoman of

In [21]:
ds_industry = pd.DataFrame(industry)
ds_industry.set_index("ticker")
ds_industry.head()
ds_industry.info()
ds_industry.to_csv('/ds_industry.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ticker           4 non-null      object
 1   company_name     4 non-null      object
 2   industry_ticker  4 non-null      object
dtypes: object(3)
memory usage: 224.0+ bytes


In [23]:
ds_key_figures = pd.DataFrame(key_figures)
ds_key_figures.set_index("ticker")
ds_key_figures.head()
ds_key_figures.info()
ds_key_figures.to_csv('/ds_key_figures.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ticker  20 non-null     object
 1   name    20 non-null     object
 2   role    20 non-null     object
dtypes: object(3)
memory usage: 608.0+ bytes
